In [1]:
from keras.models import Model
from keras.layers import Input, multiply, Dense, Reshape, dot
import tensorflow as tf
from random import uniform
%run optimizers/Optimizer.ipynb

class GradientDescentRefinement(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance,
                 numIterations=1000):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
        self.numIterations = numIterations
        self.ai, self.ri, self.ti, self.si = Optimizer.getDefaults(self.numComponents)
    
    def open(self):
        
        self.session = tf.Session()
        
        self.train_mean = tf.constant(self.mean, dtype=tf.float64)
        self.train_pcaBasis = tf.constant(self.pcaBasis, dtype=tf.float64)
        self.train_pcaVariance = tf.constant(self.pcaVariance, dtype=tf.float64)

        self.targetFeatures = tf.placeholder(tf.float64, name='targetFeatures')
        
        self.a, self.r, self.t, self.s, c = self._getVariables(self.ai, self.ri, self.ti, self.si)
        y_model = self._getTrainingModel(self.train_mean, self.train_pcaBasis, self.a, self.r, self.s, self.t, c)
        error = self._getError(self.targetFeatures, y_model, self.a, self.train_pcaVariance)
        self.optimizer = self._getOptimizer(error)
        model = tf.global_variables_initializer()
        self.session.run(model)
    
    def close(self):
        self.session.close()
    
    def optimize(self, targetFeatures):
        for _ in range(self.numIterations):
            self.session.run(self.optimizer, feed_dict={self.targetFeatures: targetFeatures})
        print('e_lmk = ' + str(self.session.run(self.e_lmk, feed_dict={self.targetFeatures: targetFeatures})))
        print('e_prior = ' + str(self.session.run(self.e_prior)))
        print('e_hyperbox = ' + str(self.session.run(self.e_hyperbox)))
        print('')
        return (np.array(self.session.run(self.a)),
                np.array(self.session.run(self.r)),
                np.array(self.session.run(self.t)),
                np.array(self.session.run(self.s))[0])
    
    def update(self, a, r, t, s):
        self.session.run(self.a.assign(a))
        self.session.run(self.r.assign(r))
        self.session.run(self.t.assign(t))
        print(s)
        self.session.run(self.s.assign(np.array([s])))
    
    def _getVariables(self, ai, ri, ti, si):
        a = tf.Variable(ai, name='a')
        r = tf.Variable(ri, name='r')
        t = tf.Variable(ti, name='t')
        s = tf.Variable(si, name='s')
        c = tf.constant(np.array([[1.,0.,0.],[0.,1.,0.]]), name='c')
        return a, r, t, s, c
    
    def _getTrainingModel(self, train_mean, train_pcaBasis, a, r, s, t, c):
        x = tf.matmul(train_pcaBasis, a)
        x = tf.add(train_mean, x)
        # Fortran Ordering!
        x = tf.transpose(tf.reshape(x,(self.numFeatures, 3)))
        x = tf.matmul(r, x)
        x = tf.matmul(c, x)
        #ts = tf.tile(t, [1, self.numFeatures])
        #ts = tf.reshape(ts, (2, self.numFeatures))
        x = (s * x) + (s * t)
        return tf.reshape(x, [-1])
    
    def _getError(self, targetFeatures, y_model, a, train_pcaVariance):
        self.e_lmk = self._getErrorLmk(targetFeatures, y_model)
        self.e_prior = self._getErrorPrior(a)
        #self.e_prior = tf.constant(1., dtype=tf.float64)
        self.e_hyperbox = self._getErrorHyperbox(a)
        '''
        return (0.33 * self.e_lmk
              + 0.33 * self.e_prior
              + 0.33 * self.e_hyperbox)
        '''
        return self.e_hyperbox
    
    def _getErrorLmk(self, targetFeatures, y_model):
        expected = tf.transpose(tf.reshape(targetFeatures, (self.numFeatures, 2)))
        actual = tf.transpose(tf.reshape(y_model, (self.numFeatures, 2)))
        return tf.cast(tf.reduce_mean(tf.reduce_sum(tf.square(expected-actual), axis=0)), tf.float64)
    
    def _getErrorPrior(self, a):
        return tf.cast(tf.square(tf.reduce_sum(tf.divide(a, tf.sqrt(self.train_pcaVariance)))), tf.float64)
    
    def _getErrorHyperbox(self, a):
        k = 3.
        w = 20
        x = a / (k * self.train_pcaVariance)
        return (tf.reduce_sum(1. / (1. + tf.math.exp(w * (1 - x))))
                + tf.reduce_sum(1. / (1. + tf.math.exp(w * (1 + x)))))
            
    def _getOptimizer(self, error):
        return tf.train.AdamOptimizer(learning_rate=0.1).minimize(error)

        